In [ ]:
from mainnet_launch.database.schema.ensure_tables_are_current.using_rebalance_plans.helper_save_rebalance_plans_to_disk import (
    read_local_rebalance_plans,
)
from mainnet_launch.constants import ALL_AUTOPOOLS, AutopoolConstants, LINEA_USD

from mainnet_launch.database.views import (
    fetch_autopool_destination_state_df,
    get_latest_rebalance_event_datetime_for_autopool,
)


import pandas as pd
import numpy as np

nov_6 = 1762315770
plans = [p for p in read_local_rebalance_plans(LINEA_USD) if p["timestamp"] >= nov_6]

for p in plans:
    p["datetime"] = pd.to_datetime(p["timestamp"], unit="s", utc=True)
    p["date"] = p["datetime"].date()


dates = sorted(set(p["date"] for p in plans))
dates
# we should have no data for 8, 9, 11
# should have data for 7, 10, 12

/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


[datetime.date(2025, 11, 5),
 datetime.date(2025, 11, 6),
 datetime.date(2025, 11, 7),
 datetime.date(2025, 11, 10),
 datetime.date(2025, 11, 12)]

In [2]:
destination_state_df = fetch_autopool_destination_state_df(LINEA_USD)

safe_tvl_by_destination = (
    (
        destination_state_df.groupby(["datetime", "readable_name"])[["autopool_implied_safe_value"]]
        .sum()
        .reset_index()
        .pivot(values="autopool_implied_safe_value", index="datetime", columns="readable_name")
    )
    .fillna(0)
    .resample("1d")
    .last()
)

backing_tvl_by_destination = (
    (
        destination_state_df.groupby(["datetime", "readable_name"])[["autopool_implied_backing_value"]]
        .sum()
        .reset_index()
        .pivot(values="autopool_implied_backing_value", index="datetime", columns="readable_name")
    )
    .fillna(0)
    .resample("1d")
    .last()
)

total_safe_tvl = safe_tvl_by_destination.sum(axis=1)
total_backing_tvl = backing_tvl_by_destination.sum(axis=1)

price_return_series = 100 * (total_backing_tvl - total_safe_tvl) / total_backing_tvl
MAX_DISPLAY_PRICE_RETURN_PERCENT = 10

price_return_series = price_return_series.where(
    (price_return_series <= MAX_DISPLAY_PRICE_RETURN_PERCENT)
    & (price_return_series >= -MAX_DISPLAY_PRICE_RETURN_PERCENT),
    np.nan,
)

portion_allocation_by_destination_df = safe_tvl_by_destination.div(total_safe_tvl, axis=0)

max_apr_by_destination = (
    (
        destination_state_df.groupby(["datetime", "readable_name"])[["unweighted_expected_apr"]]
        .max()
        .reset_index()
        .pivot(values="unweighted_expected_apr", index="datetime", columns="readable_name")
    )
    .resample("1d")
    .last()
)  # looks right

expected_return_series = (max_apr_by_destination * portion_allocation_by_destination_df).sum(axis=1)


expected_return_series = expected_return_series.replace(0, np.nan)

# total_nav_series = nav_per_share_df["NAV"]
highest_block_and_datetime = destination_state_df[["block", "datetime"]].sort_values("block", ascending=True).iloc[-1]

In [3]:
backing_tvl_by_destination

readable_name,EVK Vault eUSDC-1 (euler),EVK Vault eUSDT-1 (euler),Wrapped Aave Linea USDC (aave),Wrapped Aave Linea USDT (aave),lineaUSD (tokemak)
datetime,,,,,
2025-10-25 00:00:00+00:00,1448.777788,0.000000,0.000000,0.000000,59.999999
2025-10-26 00:00:00+00:00,1448.777788,0.000000,0.000000,0.000000,59.999999
2025-10-27 00:00:00+00:00,126014.266536,0.000000,0.000000,0.000000,0.000714
2025-10-28 00:00:00+00:00,230814.627675,0.000000,0.000000,0.000000,20291.105921
2025-10-29 00:00:00+00:00,334708.263464,0.000000,0.000000,0.000000,0.000783
2025-10-30 00:00:00+00:00,472097.403201,0.000000,0.000000,0.000000,0.000573
2025-10-31 00:00:00+00:00,546829.535954,0.000000,0.000000,0.000000,0.000301
2025-11-01 00:00:00+00:00,536485.739740,0.000000,0.000000,0.000000,1.684777
2025-11-02 00:00:00+00:00,553574.260427,0.000000,0.000000,0.000000,906.427470


In [4]:
# we have no apr data for nov 8, 9, 11
# that shoes up in the safe tvl by destination the same here as well